In [40]:
import pandas as pd
import requests
import numpy as np
import time
import re
from bs4 import BeautifulSoup
from datetime import datetime
from urllib import parse
from IPython.display import clear_output

filepath = './NewsData/mt2/'
header_list = ['언론사명', '기업명', '종목코드', '기사 업로드 날짜', '기사 제목', '해당 기사 URL', '기사 연도']

def mt_news_crwaler() :
    """
    회사 명을 키워드로 조선일보 뉴스 검색 => 기사 업로드 날짜, 기사 제목, 해당 기사 URL 크롤링하는 함수
    
    enter_df : {DataFrame} 검색할 기업의 정보가 들어있는 DataFrame
    """
    
    enter_df = pd.read_excel('./Data/분석대상기업리스트.xls', encoding = 'utf-8', dtype = {'종목코드' : object, '상장폐지일' : object})
    enter_df = enter_df[['종목코드', '기업명', '상장폐지일']]
    
    try : 
        total_df = pd.read_csv(filepath + 'mt_news_url.csv', encoding = 'utf-8', dtype = {'종목코드' : object})
    except :
        total_df = pd.DataFrame(columns= header_list)
        total_df.to_csv(filepath + 'mt_news_url.csv', encoding = 'utf-8', index = False)
    
    # 진행된 기업 제거 코드
    copy_df = enter_df.copy()
    copy_df.set_index('종목코드', inplace = True)
    search_df = copy_df.loc[set(enter_df['종목코드']) - set(total_df['종목코드'])]
    search_df.reset_index(inplace = True)
    
    sleep_time = 1 # 접근 차단을 막기위한 delay time
    sleep_page_time = 1
    
    for idx, _ in enumerate(search_df.index) :
        clear_output(wait=True)
        
        print('남은기업수 : ', len(search_df) - idx)
        
        enternum = search_df.iloc[idx]['종목코드']
        keyword = search_df.iloc[idx]['기업명']
        end_date = search_df.iloc[idx]['상장폐지일']
        journal = '머니투데이'

        if pd.isnull(end_date) :
            start_date = '20161231'
            end_date = '20191231'
        else :
            end_date = search_df.iloc[idx]['상장폐지일'].strftime('%Y%m%d')
            start_date = datetime.strptime(end_date, '%Y%m%d')
            start_date = start_date.replace(year = start_date.year - 3)
            start_date = start_date.strftime("%Y%m%d")

        curr_num = 1 # 현재 페이지 번호 
        
        start = datetime.strptime(start_date, '%Y%m%d').date()
        end = datetime.strptime(end_date, '%Y%m%d').date()
        
        is_End = False
        
        try : 
            for year in range(start.year, end.year + 1) :
                mt_list = []
                error_list = []

                # 2018년도 이후는 한번에 크롤링 끝나면 다음기업
                if is_End : break

                if year >= 2018 :
                    url = f'https://search.mt.co.kr/searchNewsList.html?srchFd=TOTAL' +\
                    f'&range=IN&category=MTNW&reSrchFlag=&preKwd=&search_type=m&kwd={keyword}&pageNum={curr_num}' +\
                    f'&bgndt=20180101&enddt=20191231&subYear=&sortType=allwordsyn&subType=mt'
                    is_End = True
                else :
                    url = f'https://search.mt.co.kr/searchNewsList.html?' +\
                    f'srchFd=TOTAL&range=TOTAL&category=MTNW&reSrchFlag=&preKwd=' +\
                    f'&search_type=m&kwd={keyword}&pageNum={curr_num}&bgndt=&enddt=&subYear={year}&sortType=allwordsyn&subType=mt'

                kwd = {'kwd' : keyword }
                params = parse.urlencode(kwd, encoding = 'EUC-KR')
                res = requests.get(url, headers = {'user-agent': 'Mozilla/5.0'}, params = params)
                soup = BeautifulSoup(res.text, 'html.parser')

                try :
                    total_page_num = int(soup.select_one('button.end').get('onclick').split('=')[-1][:-2]) # 뒤에 '; 포함되어있다
                    print('{}, {} {} {}'.format(idx, keyword, year, total_page_num))
                except Exception as e :
                    total_page_num = 0
                    print('{}, {} {} {}'.format(idx, keyword, year, total_page_num))
                    mt_list.append([journal, keyword, enternum, np.nan, np.nan, np.nan, year])
                    mt_df = pd.DataFrame(mt_list, columns= header_list)
                    mt_df.to_csv(filepath + 'mt_news_url.csv', mode = 'a', index = False, encoding = 'utf-8', header = False)
                    continue

                try :
                    for page in range(1, total_page_num + 1) :
                        print('{} {} / {}'.format(keyword, page, total_page_num))
                        
                        curr_num = page
                        
                        page_url = f'https://search.mt.co.kr/searchNewsList.html?' +\
                                f'srchFd=TOTAL&range=TOTAL&category=MTNW&reSrchFlag=&preKwd=' +\
                                f'&search_type=m&kwd={keyword}&pageNum={curr_num}&bgndt=&enddt=&subYear={year}&sortType=allwordsyn&subType=mt'
                        try :
                            kwd = {'kwd' : keyword }
                            params = parse.urlencode(kwd, encoding = 'EUC-KR')
                            res = requests.get(page_url, headers = {'user-agent': 'Mozilla/5.0'}, params = params)
                            soup = BeautifulSoup(res.text, 'html.parser')
                        except Exception as e :
                            print('page requests Error : ', e)
                            page -= 1
                            continue
                        
                        try :
                            news_list = soup.select('ul.conlist_p1 > li.bundle > div.con')

                            for news in news_list :
                                title = news.select_one('strong.subject > a').text.strip()
                                link = news.select_one('strong.subject > a').get('href')
                                time_text = news.select_one('span.etc').text.split('\xa0')[-1]
                                date = datetime.strptime(time_text, '%Y.%m.%d %H:%M').date()
                                date_year = date.year
                                
                                if start <= date and date <= end :
                                    mt_list.append([journal, keyword, enternum, time_text, title, link, date_year])
                        except Exception as e :
                            print('page Error : ', e)
                            continue
                except Exception as e :
                    print('page for loop Error : ', e)
                    print('page url : ', page_url)
                    print('=' * 60)
                    continue

                mt_df = pd.DataFrame(mt_list, columns= header_list)
                mt_df.to_csv(filepath + 'mt_news_url.csv', mode = 'a', index = False, encoding = 'utf-8', header = False)

                # 시간 딜레이
                time.sleep(sleep_time)
        except Exception as e :
            print('year for loop Error : ', e)
            time.sleep(5) 
            # 5초 쉬고 다시 시작
            year -= 1
            continue

In [41]:
mt_news_crwaler()

남은기업수 :  1
166, 바이나믹 2007 0
166, 바이나믹 2008 0
166, 바이나믹 2009 0
166, 바이나믹 2010 1
바이나믹 1 / 1


### 크롤링된 데이터셋 중복 제거 후 저장

In [42]:
filepath = './NewsData/mt2/'
crawl_df = pd.read_csv(filepath + 'mt_news_url.csv', encoding = 'utf-8', dtype = {'종목코드' : object})

In [43]:
crawl_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1354277 entries, 0 to 1354276
Data columns (total 7 columns):
언론사명         1354277 non-null object
기업명          1354277 non-null object
종목코드         1354277 non-null object
기사 업로드 날짜    1353799 non-null object
기사 제목        1353799 non-null object
해당 기사 URL    1353799 non-null object
기사 연도        1354277 non-null int64
dtypes: int64(1), object(6)
memory usage: 72.3+ MB


In [10]:
crawl_df.head()

,언론사명,기업명,종목코드,기사 업로드 날짜,기사 제목,해당 기사 URL,기사 연도
0,머니투데이,굿모닝신한증권,008670,2003-01-01,"""1분기 600찍고 4분기 1000 도전""",https://news.mt.co.kr/mtview.php?no=2002123117...,2003
1,머니투데이,굿모닝신한증권,008670,2002-12-30,올 증시 `음봉마감`,https://news.mt.co.kr/mtview.php?no=2002123017...,2002
2,머니투데이,굿모닝신한증권,008670,2002-12-30,"[개장전]폐장일, 유종의 미 거둘까",https://news.mt.co.kr/mtview.php?no=2002123008...,2002
3,머니투데이,굿모닝신한증권,008670,2002-12-27,"불안한 증시, 루머에 휘청",https://news.mt.co.kr/mtview.php?no=2002122714...,2002
4,머니투데이,굿모닝신한증권,008670,2002-12-27,애널리스트 위법거래 무더기 적발,https://news.mt.co.kr/mtview.php?no=2002122713...,2002


In [44]:
len(crawl_df['종목코드'].unique())

2709

In [11]:
test_df = crawl_df.drop_duplicates(['언론사명', '기업명', '종목코드', '기사 업로드 날짜', '기사 제목', '해당 기사 URL', '기사 연도'])

In [12]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1344193 entries, 0 to 1357770
Data columns (total 7 columns):
언론사명         1344193 non-null object
기업명          1344193 non-null object
종목코드         1344193 non-null object
기사 업로드 날짜    1344193 non-null object
기사 제목        1344193 non-null object
해당 기사 URL    1344193 non-null object
기사 연도        1344193 non-null int64
dtypes: int64(1), object(6)
memory usage: 82.0+ MB


In [13]:
test_df.head()

,언론사명,기업명,종목코드,기사 업로드 날짜,기사 제목,해당 기사 URL,기사 연도
0,머니투데이,굿모닝신한증권,008670,2003-01-01,"""1분기 600찍고 4분기 1000 도전""",https://news.mt.co.kr/mtview.php?no=2002123117...,2003
1,머니투데이,굿모닝신한증권,008670,2002-12-30,올 증시 `음봉마감`,https://news.mt.co.kr/mtview.php?no=2002123017...,2002
2,머니투데이,굿모닝신한증권,008670,2002-12-30,"[개장전]폐장일, 유종의 미 거둘까",https://news.mt.co.kr/mtview.php?no=2002123008...,2002
3,머니투데이,굿모닝신한증권,008670,2002-12-27,"불안한 증시, 루머에 휘청",https://news.mt.co.kr/mtview.php?no=2002122714...,2002
4,머니투데이,굿모닝신한증권,008670,2002-12-27,애널리스트 위법거래 무더기 적발,https://news.mt.co.kr/mtview.php?no=2002122713...,2002


In [14]:
test_df.to_csv(filepath + 'mt_news_url.csv', index = False, encoding = 'utf-8')

In [16]:
len(test_df['종목코드'].unique())

2540

In [20]:
bind_df = pd.read_csv(filepath + 'mt_news_url_bind.csv', encoding = 'utf-8', dtype = {'종목코드' : object})

In [21]:
bind_df.head()

,언론사명,기업명,종목코드,기사 업로드 날짜,기사 제목,해당 기사 URL,기사 연도
0,머니투데이,보광티에스,066690,NaN,NaN,NaN,2009
1,머니투데이,보광티에스,066690,NaN,NaN,NaN,2010
2,머니투데이,무림P&P,009580,NaN,NaN,NaN,2016
3,머니투데이,무림P&P,009580,NaN,NaN,NaN,2017
4,머니투데이,허메스홀딩스,012400,NaN,NaN,NaN,2009


In [22]:
con_df = pd.concat([test_df, bind_df])

In [24]:
len(con_df['종목코드'].unique())

2542

In [29]:
con_df.sort_values('종목코드', inplace = True)

In [31]:
con_df.reset_index(drop = True, inplace = True)

In [32]:
con_df.to_csv(filepath + 'mt_news_url.csv', index = False, encoding = 'utf-8')

## Test